# 창의설계축전 2023
> 제목: 어르신들을 위한 원격 장기 및 인터넷 장기 연동
>
> 팀명: 탑골공원
> 
> 팀원: 이현진, 김건우, 이성환


>Setup

In [ ]:
import os
import pygame

import serial
import time
import cv2
import numpy as np
import pickle

# __init__
from janggi import constants
from janggi.base.board import Board
from janggi.base.camp import Camp
from janggi.base.formation import Formation
from janggi.base.location import Location
from janggi.base.move import MoveSet
from janggi.base.piece import Piece, PieceType
from janggi.game.janggi_game import JanggiGame
from janggi.game.game_log import GameLog
from janggi.ui.game_window import GameWindow
from janggi.ui.replay_viewer import ReplayViewer
from janggi.proto import log_pb2
from janggi.utils import generate_random_game, play, replay

# constants
MIN_ROW = 0
MAX_ROW = 9
MIN_COL = 0
MAX_COL = 8
NUM_ROWS = MAX_ROW - MIN_ROW + 1
NUM_COLS = MAX_COL - MIN_COL + 1

CASTLE_MIN_COL = 3
CASTLE_MAX_COL = 5
CASTLE_TOP_MIN_ROW = 0
CASTLE_TOP_MAX_ROW = 2
CASTLE_BOT_MIN_ROW = 7
CASTLE_BOT_MAX_ROW = 9

CASTLE_TOP_CENTER = (1, 4)
CASTLE_BOT_CENTER = (8, 4)

HAN_ADVANTAGE = 1.5

# utils
import logging
import random

from janggi.base.camp import Camp
from janggi.base.formation import Formation
from janggi.game.janggi_game import JanggiGame
from janggi.game.game_log import GameLog
from janggi.ui.game_player import GamePlayer
from janggi.ui.replay_viewer import ReplayViewer
from janggi.proto import log_pb2

logging.basicConfig()
logging.root.setLevel(logging.DEBUG)


def replay(filepath: str):
    """
    Replay a game by parsing the log file at the given path.

    Args:
        filepath (str): Path of the proto-serialized log file.
    """
    log_file = open(filepath, "rb")
    log_proto = log_pb2.Log()
    log_proto.ParseFromString(log_file.read())
    game_log = GameLog.from_proto(log_proto)
    game_log.generate_board_log()
    replay_viewer = ReplayViewer(game_log)
    replay_viewer.run()


def play(game: JanggiGame):
    """
    Play a game by running GamePlayer.

    Args:
        game (JanggiGame): Pre-initialized game to play.
    """
    player = GamePlayer(game)
    player.run()


def generate_random_game():
    """Generate a random Janggi game."""
    camp = Camp(random.choice([-1, 1]))
    cho_formation = Formation(random.randint(1, 4))
    han_formation = Formation(random.randint(1, 4))
    return JanggiGame(camp, cho_formation, han_formation)

>게임 장기와 플로터 장기의 연동(플로터 장기 코드)

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import serial

cred = credentials.Certificate('PATH TO YOUR KEY')
firebase_admin.initialize_app(cred,{
    'databaseURL' : 'URL OF YOUR DB'
})
dir = db.reference('HAN')
ref = db.reference()

In [ ]:
ser = serial.Serial(
   port='COM5',
   baudrate=115200,
)

myColor = input("which color? red(r) or green(g)?:")

In [3]:
import cv2
import numpy as np
import time

#[옮기기전 행, 열, 옮긴 후 행, 열]  <= 길이가 4인 배열로 파일에 pickle.dump 했습니다; Line 100, 106, 124, 130
# q 버튼 누르면 영상 중지됩니다

frameWidth = 640
frameHeight = 480
#cap = cv2.VideoCapture('edit2.mp4') #영상
#cap = cv2.VideoCapture(0)   #웹캠
cap = cv2.VideoCapture(1)   #DroidCam으로 연결한 카메라
cap.set(3, frameWidth)
cap.set(4, frameHeight)


def empty(a):
    pass
cv2.namedWindow("result")
cv2.createTrackbar('threshold', "result", 217, 255, empty) #돌 인식하기 위해 적절한 값 설정
cv2.createTrackbar('velocity', "result", 1, 1000, empty) #영상 속도(프레임 간의 대기 시간 변수여서 작은 값일수록 빠름)
cv2.createTrackbar("Area", "result", 200, 10000, empty) #너무 작은 contour는 무시하기 위한 변수



#여러 이미지를 하나의 창에 보여주는 함수(테스트용)
def stackImage(scale, imgArray):
    rows=len(imgArray)
    cols=len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range(rows):
            for y in range(cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0,0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1],imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y] = cv2.cvtColor(imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0,0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1],imgArray[0].shape[0]), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x]= cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor = np.hstack(imgArray)
        ver = hor
    return ver


#말 인식 후 stoneArray에 중심의(x,y)좌표 append
def getContours(img, imgContour):
    global stoneArray
    stoneArray=[]
    contours, hierarchy = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        areaMin = cv2.getTrackbarPos("Area", "result")
        if area>areaMin:
            cv2.drawContours(imgContour, cnt, -1, (255,0,255), 2)
            peri = cv2.arcLength(cnt, False)
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            x, y, w, h = cv2.boundingRect(approx)
            if len(approx)>=0: 
                centerX=x+w//2
                centerY=y+h//2
                stoneArray.append((centerX,centerY))
                
                sumR=np.sum(imgContour[centerY-5:centerY+5, centerX-5:centerX+5,2])
                
                cv2.putText(imgContour, str(int(sumR)),(x,y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255,0,0),1)
                
                cv2.rectangle(imgContour, (x,y), (x+w, y+h), (0,255,0), 2)
                cv2.circle(imgContour, (centerX, centerY), 3, (0,0,255), -1, cv2.LINE_AA)
                cv2.putText(imgContour, str(len(approx)),(x+w,y+10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,255,0),1)
                cv2.putText(imgContour, str(int(area)),(x+w,y+20), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,255,0),1)
                #cv2.putText(imgContour, '{0},{1}'.format(x+w/2,y+h/2),(x-w/2,y+h+20 if y<img.shape[1]/2 else y-5), cv2.FONT_HERSHEY_COMPLEX, 0.4, (0,255,0),1)


#돌 움직였는지 판단 후 좌표 출력
def moveStone():
    global board, board_before, turn, gStone, rStone, moveDone, prev
    beforePos=(0,0)
    afterPos=(0,0)
    cntG = sum([list(i).count('g') for i in board])
    cntR = sum([list(i).count('r') for i in board])
    if turn=='g' and cntG==gStone:
        for i in range(10):
            for j in range(9):
                if board_before[i][j]!='g' and board[i][j]=='g':
                    afterPos=(i,j)
                if board_before[i][j]=='g' and board[i][j]!='g':
                    beforePos=(i,j)
        if beforePos[0]!=afterPos[0] or beforePos[1]!=afterPos[1]:
            if cntR==rStone:
                """
                이두이노, 서버로 (beforePos[0], beforePos[1], afterPos[0], afterPos[1]) 전송
                """
                ser.write('g m {0} {1} {2} {3}'.format(9-beforePos[0], 8-beforePos[1], 9-afterPos[0], 8-afterPos[1]).encode())
                ref.update({'CHO' : '{0} {1} {2} {3}'.format(9-beforePos[0], 8-beforePos[1], 9-afterPos[0], 8-afterPos[1])})
                print("g moved from {0} to {1}".format(beforePos, afterPos))
                turn='r'
                print(board)
                board_before=board.copy()
            if cntR==rStone-1 and board_before[afterPos[0]][afterPos[1]]=='r':
                """
                이두이노,서버로 (beforePos[0], beforePos[1], afterPos[0], afterPos[1]) 전송
                """
                ser.write('g t {0} {1} {2} {3}'.format(9-beforePos[0], 8-beforePos[1], 9-afterPos[0], 8-afterPos[1]).encode())
                ref.update({'CHO' : '{0} {1} {2} {3}'.format(9-beforePos[0], 8-beforePos[1], 9-afterPos[0], 8-afterPos[1])})
                print("g moved from {0} to {1}".format(beforePos, afterPos))
                print('g took r')
                rStone-=1
                turn='r'
                print(board)
                board_before=board.copy()
        
            
    elif turn=='r' and cntR==rStone:
        if dir.get()!=prev:
            prev=dir.get()
            if board[9-int(dir.get()[4])][8-int(dir.get()[6])]=='g':
                ser.write('r t {0} {1} {2} {3}'.format(dir.get()[0], dir.get()[2], dir.get()[4], dir.get()[6]).encode())
            else:
                ser.write('r m {0} {1} {2} {3}'.format(dir.get()[0], dir.get()[2], dir.get()[4], dir.get()[6]).encode())
            """
            아두이노로 (dir.get()[0], dir.get()[2], dir.get()[4], dir.get()[6]) 전송
            """
            
            moveDone = ser.readline()=='Done'
            if board[9-int(dir.get()[4])][8-int(dir.get()[6])]=='g':
                print("r moved from ({0}, {1}) to ({2}, {3})".format(9-int(dir.get()[0]), 8-int(dir.get()[2]), 9-int(dir.get()[4]), 8-int(dir.get()[6])))
                print('r took g')
                gStone-=1
                turn='g'
                board[9-int(dir.get()[4])][8-int(dir.get()[6])]='r'
                board[9-int(dir.get()[0])][8-int(dir.get()[2])]=' '
                print(board)
                board_before=board.copy()
            else:
                print("r moved from ({0}, {1}) to ({2}, {3})".format(9-int(dir.get()[0]), 8-int(dir.get()[2]), 9-int(dir.get()[4]), 8-int(dir.get()[6])))
                turn='g'
                board[9-int(dir.get()[4])][8-int(dir.get()[6])]='r'
                board[9-int(dir.get()[0])][8-int(dir.get()[2])]=' '
                print(board)
                board_before=board.copy()
            
        time.sleep(0.5)
        
                


#판의 경계와 격자 위치는 시작할 때 한 번만 설정
moveDone=False
gStone=16
rStone=16
prev='0 0 0 0'
t=0
turn='g'
critical=19500
while t<10:
    t+=1
    ret, img = cap.read()
    if not ret:
        break
    if cv2.waitKey(1) == ord('q'):
        break 
    
    #img 전처리
    img=cv2.resize(img, None, fx=0.82, fy=1)
    imgContour = img.copy()
    threshold=cv2.getTrackbarPos('threshold', 'result')
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(imgGray, threshold, 255, cv2.THRESH_BINARY)
    binary = cv2.dilate(binary, np.ones((7,7)), iterations=1)
    
    getContours(binary, imgContour)
    
    
    #격자(빨간 줄) 설정 위한 경계(r,l,u,d) 설정
    right=0
    left=1000
    down=0
    up=1000
    for centerX, centerY in stoneArray:
        right=np.max([right,centerX])
        left=np.min([left,centerX])
        down=np.max([down,centerY])
        up=np.min([up,centerY])
    
    
    #격자 설정
    deltaX=(right-left)/8
    deltaY=(down-up)/9
    xLines=[]
    yLines=[]
    for i in range(10):
        X=int(left-deltaX/2+i*deltaX)
        cv2.line(imgContour, (X, int(up-deltaY/2)), (X, int(down+deltaY/2)), (0,0,255), 1, cv2.LINE_AA)
        xLines.append(X)
    for i in range(11):
        Y=int(up-deltaY/2+i*deltaY)
        cv2.line(imgContour, (int(left-deltaX/2), Y), (int(right+deltaX/2),Y), (0,0,255), 1, cv2.LINE_AA)
        yLines.append(Y)        
    
    #초기 판 상태 설정
    board_before = np.full((10,9), ' ')
    for centerX, centerY in stoneArray:
        for i in range(9):
            for j in range(10):
                if xLines[i]<centerX and centerX<xLines[i+1] and yLines[j]<centerY and centerY<yLines[j+1]:
                    sumG=np.sum(img[centerY-5:centerY+5, centerX-5:centerX+5,1])
                    sumR=np.sum(img[centerY-5:centerY+5, centerX-5:centerX+5,2])
                    board_before[j][i]= 'r' if sumR>critical else 'g'

print(board_before)
#main
t=0
while True:
    ret, img = cap.read()
    if not ret:
        break
        
    #이미지 전처리
    img=cv2.resize(img, None, fx=0.82, fy=1)
    imgContour = img.copy()
    threshold=cv2.getTrackbarPos('threshold', 'result')
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(imgGray, threshold, 255, cv2.THRESH_BINARY)
    binary = cv2.dilate(binary, np.ones((3,3)), iterations=1)
    getContours(binary, imgContour)
    
    
    #img 위에 경계 표시
    for i in xLines:
        cv2.line(imgContour, (i, int(up-deltaY/2)), (i, int(down+deltaY/2)), (0,0,255), 1, cv2.LINE_AA)
    for i in yLines:
        cv2.line(imgContour, (int(left-deltaX/2), i), (int(right+deltaX/2),i), (0,0,255), 1, cv2.LINE_AA)
    
    
    #getContours에서 인식한 말을 board에 위치, 색 표시
    board = np.full((10,9), ' ')
    for centerX, centerY in stoneArray:
        for i in range(9):
            for j in range(10):
                if xLines[i]<centerX and centerX<xLines[i+1] and yLines[j]<centerY and centerY<yLines[j+1]:
                    sumG=np.sum(img[centerY-5:centerY+5, centerX-5:centerX+5,1])
                    sumR=np.sum(img[centerY-5:centerY+5, centerX-5:centerX+5,2])
                    board[j][i]= 'r' if sumR>critical else 'g'

    moveStone()

    #img 출력
    imgStack = stackImage(1.2, ([imgContour,binary]))
    cv2.imshow("result", imgStack)
    #cv2.imshow("result", imgContour)
    velocity=cv2.getTrackbarPos('velocity', 'result')
    if cv2.waitKey(velocity) == ord('q'):
        break
    
    

cap.release()
cv2.destroyAllWindows()

[['r' 'r' 'r' 'r' ' ' 'r' 'r' 'r' 'r']
 [' ' ' ' ' ' ' ' 'r' ' ' ' ' ' ' ' ']
 [' ' 'r' ' ' ' ' ' ' ' ' ' ' 'r' ' ']
 ['r' ' ' 'r' ' ' 'r' ' ' 'r' ' ' 'r']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['g' ' ' 'g' ' ' 'g' ' ' 'g' ' ' 'g']
 [' ' 'g' ' ' ' ' ' ' ' ' ' ' 'g' ' ']
 [' ' ' ' ' ' ' ' 'g' ' ' ' ' ' ' ' ']
 ['g' 'g' 'g' 'g' ' ' 'g' 'g' 'g' 'g']]


>게임 장기와 플로터 장기의 연동(게임 장기 코드)

In [ ]:
import logging
import random
import time

from janggi import Camp
from janggi import Formation
from janggi import Location
from janggi import GameLog
from janggi import JanggiGame
from janggi import GameWindow
from janggi import ReplayViewer

import firebase_admin
from firebase_admin import credentials
from firebase_admin import db



logging.basicConfig()
logging.root.setLevel(logging.DEBUG)

if __name__ == '__main__':
    # camp = Camp(random.choice([-1, 1]))
    camp = Camp(1)
    # cho: 1
    # han: -1
    
    cho_formation = Formation(4)
    han_formation = Formation(4)
    # 마상상마: 1
    # 상마마상: 2
    # 상마상마: 3
    # 마상마상: 4
    
    game = JanggiGame(camp, cho_formation, han_formation)
    print(f"cho: {game.cho_score} / han: {game.han_score}")
    print(game.board)

    play(game)
 
    # Test replay viewer
    # moves = [(Location(3, 0), Location(3, 1)), (Location(6, 8),
    #                                             Location(6, 7)), (Location(0, 0), Location(4, 0))]
    # game_log = GameLog(cho_formation, han_formation, camp, moves)
    # game_log.generate_board_log()
    # replay_viewer = ReplayViewer(game_log)
    # replay_viewer.run()
